In [4]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import time
import gc
# from tqdm import tqdm

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [14]:
pred1 = pd.read_csv('pred_model1.csv')
pred2 = pd.read_csv('pred_model2.csv')
pred3 = pd.read_csv('pred_model3.csv')

pred1.shape
pred2.shape
pred3.shape

(269737, 2)

(1077175, 2)

(269737, 3)

In [15]:
pred = pred1.merge(pred2, on=['instance_id'], how='left')
pred = pred.merge(pred3, on=['instance_id'], how='left')

del pred1
del pred2
del pred3
gc.collect()

pred.shape

1103

(269737, 5)

In [16]:
pred.columns = ['instance_id', 'pred1', 'pred2', 'pred3', 'is_trade']
pred.head()

,instance_id,pred1,pred2,pred3,is_trade
0,9026430732534853093,0.004561,0.003338,0.002570,0
1,9155155901436899292,0.031189,0.012083,0.000627,0
2,3094163606679545517,0.022309,0.007001,0.000848,0
3,8573303376462104822,0.122812,0.054428,0.021033,0
4,3578285614312748801,0.043221,0.000757,0.000127,0


In [17]:
from sklearn.metrics import log_loss

In [21]:
%%time
best_logloss = 1
best_i = 0

for i in range(0, 1000, 25):
    i = i / 1000
    tmp = i * pred.pred1 + (1 - i) * pred.pred2
    tmp_loss = log_loss(pred.is_trade, tmp)
    if tmp_loss < best_logloss:
        best_logloss = tmp_loss
        best_i = i
            
print(str(best_logloss) + '->' + str(best_i) + ':' + str(1 - best_i))

0.16360208795625725->0.9:0.09999999999999998
CPU times: user 3.1 s, sys: 20 ms, total: 3.12 s
Wall time: 2.97 s


In [34]:
%%time
best_logloss = 1
best_i = 0
best_j = 0

for i in range(0, 1001, 25):
    for j in range(0, 1001-i, 25):
        tmp = i/1000 * pred.pred1 + j/1000 * pred.pred2 + (1 - i/1000 - j/1000) * pred.pred3
        tmp_loss = log_loss(pred.is_trade, tmp)
        if tmp_loss < best_logloss:
            best_logloss = tmp_loss
            best_i = i
            best_j = j
            
print(str(best_logloss) + '->' + str(best_i) + ':' + str(best_j) + ':' + str(1000 - best_i - best_j))

0.16360208795625725->900:100:0
CPU times: user 1min 8s, sys: 488 ms, total: 1min 8s
Wall time: 1min 4s
